In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel('all stocks50 days.xlsx', index_col=0)

In [3]:
df_copy = df.copy()
df_copy = df_copy.drop(columns=['Capital Gains'])
df_copy = df_copy.drop(columns=['Adj Close'])

In [4]:
indexes = df_copy.Symbol.value_counts().loc[lambda x: x>=50].index  #get indexes that have 50 or more entries
df_g50 = df_copy[df_copy['Symbol'].isin(indexes)] #put only those indexes into df

In [5]:
#this cell runs slow - i will buy you a cookie if you can speed it up

reg = LinearRegression()
df_g50['High slope 20 days'] = 0
df_g50['Close slope 20 days'] = 0
df_g50['High Linear reg MSE'] = 0
df_g50['Close Linear reg MSE'] = 0

X_train = np.arange(0, 20, 1).reshape(-1, 1)
X_test = np.arange(20, 25, 1).reshape(-1, 1)

unique_symbols = df_g50['Symbol'].unique()

#for index in range(5):   #for testing - leave in place
 #   symbol = unique_symbols[index]

#predict on the first 20 and test on the same - in future versions these will be learned params
for symbol in unique_symbols:
    subset = df_g50[df_g50['Symbol'] == symbol].iloc[0:20, :]
    Y1 = subset['High']
    Y2 = subset['Close']
    
    model1 = reg.fit(X_train, Y1)
    prediction1 = model1.predict(X_train)
    mse1 = mean_squared_error(Y1, prediction1)
    
    model2 = reg.fit(X_train, Y2)
    prediction2 = model2.predict(X_train)
    mse2 = mean_squared_error(Y2, prediction2)
    
    subset = df_g50[df_g50['Symbol'] == symbol]
    df_g50.loc[subset.index, 'High slope 20 days'] = model1.coef_[0]
    df_g50.loc[subset.index, 'Close slope 20 days'] = model2.coef_[0]
    df_g50.loc[subset.index, 'High Linear reg MSE'] = mse1
    df_g50.loc[subset.index, 'Close Linear reg MSE'] = mse2
    
    
#display(df_g50)

In [6]:
df_b20 = df_g50.groupby('Symbol').apply(lambda group: group.iloc[20:, :])
stocks_to_buy = df_b20[df_b20['Volume'] > 0]


indexes = stocks_to_buy.Symbol.value_counts().loc[lambda x: x>=30].index  
stocks_to_buy = stocks_to_buy[stocks_to_buy['Symbol'].isin(indexes)] 

In [7]:
display(stocks_to_buy)
stocks_to_buy.to_csv('stocks to buy.csv', index=True)

Open        High         Low       Close   Volume Symbol  \
Symbol                                                                          
A      20      143.770004  144.850006  142.789993  143.729996  1211200      A   
       21      145.119995  145.259995  141.130005  141.339996  1183900      A   
       22      144.179993  145.500000  143.429993  144.119995  1655800      A   
       23      143.800003  145.789993  143.009995  144.460007  1247300      A   
       24      146.000000  147.429993  145.500000  147.399994   873700      A   
...                   ...         ...         ...         ...      ...    ...   
ZYXI   318705   10.830000   10.880000   10.650000   10.730000    64600   ZYXI   
       318706   10.770000   10.810000   10.310000   10.340000   152600   ZYXI   
       318707   10.440000   10.780000   10.430000   10.490000   162000   ZYXI   
       318708   10.490000   10.680000   10.400000   10.680000    99800   ZYXI   
       318709   11.030000   11.030000   10.725000   10.860000    92404   ZYXI   

               High slope 20 days  Close slope 20 days  High Linear reg MSE  \
Symbol                                                                        
A      20               -0.079653            -0.079653             1.793944   
       21               -0.079653            -0.079653             1.793944   
       22               -0.079653            -0.079653             1.793944   
       23               -0.079653            -0.079653             1.793944   
       24               -0.079653            -0.079653             1.793944   
...                           ...                  ...                  ...   
ZYXI   318705           -0.013316            -0.013316             0.029991   
       318706           -0.013316            -0.013316             0.029991   
       318707           -0.013316            -0.013316             0.029991   
       318708           -0.013316            -0.013316             0.029991   
       318709           -0.013316            -0.013316             0.029991   

               Close Linear reg MSE  
Symbol                               
A      20                  1.785066  
       21                  1.785066  
       22                  1.785066  
       23                  1.785066  
       24                  1.785066  
...                             ...  
ZYXI   318705              0.038187  
       318706              0.038187  
       318707              0.038187  
       318708              0.038187  
       318709              0.038187  

[174120 rows x 10 columns]